In [3]:
pip install natasha

In [10]:
import json
import pathlib
import typing as tp
from natasha import (
    Segmenter,
    NewsNERTagger,
    NewsEmbedding,
    Doc
)
from pymorphy2 import MorphAnalyzer

In [11]:
import pickle

In [12]:
with open('data/sentiment_texts.pickle', 'rb') as f1:
    sentiment_texts = pickle.load(f1)

In [13]:
def normalize_company_name(name):
    morph = MorphAnalyzer()
    normalized_words = []
    for word in name.split():
        parsed_word = morph.parse(word)[0]
        normalized_word = parsed_word.inflect({'nomn', 'sing'})
        if normalized_word:
            normalized_words.append(normalized_word.word)
        else:
            normalized_words.append(word)
    return ' '.join(normalized_words)

In [14]:
def extract_company_names(text):
    segmenter = Segmenter()
    emb = NewsEmbedding()
    ner_tagger = NewsNERTagger(emb)

    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)

    companies = [span.text for span in doc.spans if span.type == 'ORG']
    return [normalize_company_name(company) for company in companies]


In [16]:
def score_texts(texts):
    results = []
    i = 0
    for text in texts:
        print(i)
        i+=1
        company_names = extract_company_names(text)
        company_names_str = ', '.join(company_names) if company_names else ""
        results.append(f"[{company_names_str}] {text}")
    return results

In [17]:
scores = score_texts(sentiment_texts['MessageText'])
print(scores)

0
1
2
3
4
5


KeyboardInterrupt: 

In [ ]:
from deeppavlov import build_model, configs

def sentiment_model():
    model = build_model(configs.classifiers.rusentiment_convers_bert, download=True)
    return model

model = sentiment_model()

In [18]:
def analyze_sentiment(text):
    return model([text])[0]


In [19]:
def score_texts(texts):
    results = []
    for text in texts:
        company_names = extract_company_names(text)
        sentiment = analyze_sentiment(text)
        company_names_str = ', '.join(company_names) if company_names else "No company names"
        results.append(f"Sentiment: {sentiment}, Companies: {company_names_str}, Text: {text}")
    return results

In [ ]:
pip install dostoevsky

In [ ]:
sentiment_texts

In [20]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

tokenizer = RegexTokenizer()
FastTextSocialNetworkModel.MODEL_PATH = 'data/fasttext-social-network-model.bin'
model = FastTextSocialNetworkModel(tokenizer=tokenizer)
sentiment_list = []
results = model.predict(sentiment_texts['MessageText'], k=2)
for sentiment in results:
    sentiment_list.append(sentiment)

In [21]:
print(sentiment_list[3])

{'neutral': 0.8080772161483765, 'negative': 0.16027602553367615}


In [22]:
keys = set()
for i in sentiment_list:
  for key, value in i.items():
    keys.add(key)
print(keys)

{'positive', 'neutral', 'negative', 'skip', 'speech'}


In [23]:
import math
from collections import Counter

In [24]:
def sentiment_score(sentiment_data):
    weights = {'negative': 1, 'neutral': 3.5, 'positive': 5, 'skip':0, 'speech':0}
    scores = []
    count = 0
    for data in sentiment_data:
        score = sum(data.get(key, 0) * weight for key, weight in weights.items())
        scores.append(min(5, math.ceil(score)))
        count += 1

    return scores

In [27]:
sen_score = sentiment_score(sentiment_list)
print(sen_score[3])

3


In [26]:
print(Counter(sentiment_score(sentiment_list)))

Counter({3: 5635, 4: 2819, 2: 756, 1: 64, 5: 15})


In [30]:
print(len(sen_score))
real_score = sentiment_texts.SentimentScore
print(real_score.shape)


9289
(9289,)


In [32]:
from sklearn.metrics import accuracy_score
ac = accuracy_score(sen_score, real_score)
ac

0.3476154591452255